**Clinical Recommendation System**

Recommendation system takes MRI image from an input folder, incorporates the developed model to make a classification and then provides a clinical support recommendation depending on the classification.

- Supports batch MRI input and batch output (in the form of a CSV file with name of MRI and associated recommendation)

- Improves automatically as model is improved over time as retrieves model and weightings from online drive folder

**Author & References**

Code is all authored by Ben Fleming

Libraries used:

Tensorflow

Keras

Numpy

Csv

Datetime

Pandas

**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Load Relevant Libraries**

In [ ]:
#libraries used can be seen in this section

import tensorflow as tf
import numpy as np
from tensorflow import keras
import csv
from datetime import date
import pandas

#Verify versions are correct

tf.__version__
keras.__version__

**Load Model**

In [ ]:
#Load the model from the saved filepath

filepath = '/content/drive/My Drive/AD Project/Binary CNNs/Binary Model Saves/VGG16-Binary'

loadedModel = tf.keras.models.load_model(filepath, custom_objects=None, compile=False, options=None)

#Examine model

loadedModel.summary()

**Load Recommendations from File**

In [ ]:
  recommendations = ['Medical practitioner: Conduct mental ability test assessing memory retention, attention span, language and visuospatial abilities to further clarify severity of cognitive impairment', 
                     'Medical practitioner: No Alzheimers associated deterioration detected']

  classPredictions = ['Alzheimers Disease or Mild Cognitive Impairment', 'Cognitively Normal']

**Load Files from Folder**

In [ ]:
#Keep the same folder for retrieving MRI images

rootDirectory = '/content/drive/My Drive/AD Project/Recommendation System/MRI Drop Folder'

#Create data generator for supplying test images to the model

predictGenerator = tf.keras.preprocessing.image.ImageDataGenerator()

#Flow images to prediction

flowPredictGenerator = predictGenerator.flow_from_directory(rootDirectory, target_size=(224,224), color_mode='rgb',shuffle=False,interpolation="nearest",batch_size=1, class_mode=None)


**Use Model To Iteratively Make Predictions On Given Images**

In [ ]:
predictions = np.argmax(loadedModel.predict(flowPredictGenerator), axis=-1)

fileNames = flowPredictGenerator.filenames

print(fileNames)

print(predictions)

#strip filenames to new array

strippedFilenames = []

for i in fileNames:
  splitString = i.split("/",3)
  strippedFilenames.append(splitString[-1])

print(strippedFilenames)



In [ ]:
#Create list containing recommendations 

recommendationsCSV = []

for i in predictions:
  if (i == 0):
      recommendationsCSV.append(recommendations[0])
  if (i == 1):
      recommendationsCSV.append(recommendations[1])

print(recommendationsCSV)


In [ ]:
classpredictionCSV = []

for i in predictions:
  if (i == 0):
      classpredictionCSV.append(classPredictions[0])
  if (i == 1):
      classpredictionCSV.append(classPredictions[1])

print(classpredictionCSV)

In [ ]:
#Create nested list for csv

csvWrite = []

csvWrite.append(['Filename', 'Classification', 'Recommendation'])

for i in strippedFilenames:
  tempList = []
  tempList.append(strippedFilenames[strippedFilenames.index(i)])
  tempList.append(classpredictionCSV[strippedFilenames.index(i)])
  tempList.append(recommendationsCSV[strippedFilenames.index(i)])
  csvWrite.append(tempList)

print(csvWrite)


**Write Filename, Prediction and Recommendation to CSV**

Columns:

- Filename
- Classification
- Recommendation

In [ ]:
#Create new csv

reportDirectory = '/content/drive/My Drive/AD Project/Recommendation System/Reports/'

today = date.today()

csvFileName =  reportDirectory + today.strftime("%d-%m-%Y") + 'BatchRecommendationReport.csv'

print(csvFileName)

with open(csvFileName, 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerows(csvWrite)



**Display CSV**

In [ ]:
#Read and print last created csv file

df = pandas.read_csv(csvFileName)

display(df)